In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!nvidia-smi --query-gpu=name --format=csv,noheader
!pip install dhash

Tesla T4
  Created wheel for dhash: filename=dhash-1.3-cp36-none-any.whl size=7274 sha256=54a9c79855f8392a27b0621b494a08f58d039a3f64a7fb2deea309e06aef517e
  Stored in directory: /root/.cache/pip/wheels/a4/cb/88/d7cf61a1ac52042d59cc0c7a130c04b1541dfd439570c6ea41
Successfully built dhash


In [0]:
# creaz dataset

!rm -rf sep_eps
!rm -rf ds
!mkdir ds

from google.colab.patches import cv2_imshow

import cv2
import dhash
from PIL import Image
import os
import shutil

def get_num_bits_different(hash1, hash2):
    return bin(hash1 ^ hash2).count('1')

vids = ['at_s01e01.mp4', 'at_s01e02.mp4', 'at_s01e03.mp4', 'at_s01e04.mp4', 'sao_e01.mkv', 'Akira.mkv']
#vids = ['at_s01e01.mp4', 'at_s01e02.mp4', 'at_s01e03.mp4', 'sao_e01.mkv', 'Akira.mkv']

# used to save ds
fld = 0

for vid_path in vids:

  cap = cv2.VideoCapture('/content/drive/My Drive/__progetto int sys/data/' + vid_path)
  vid_name = vid_path.split('.')[0]
  totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  # 60k per f
  if totalFrames > 4000:
    totalFrames = 4000
  if vid_name == 'at_s01e01' or vid_name == 'at_s01e02' or vid_name == 'at_s01e03' or vid_name == 'at_s01e04':
    totalFrames = 1000
  main_dir = 'sep_eps/' + vid_name
  if os.path.exists(main_dir):
      shutil.rmtree(main_dir)
  os.makedirs(main_dir)

  # 0 - 2: frame ripetuto => non lo salvo
  # 0 - 9 per Akira
  thresh_rip = 2
  if vid_name == 'Akira':
    thresh_rip = 9
    # skip 2400 frames
    for _ in range(2400):
      _, _ = cap.read()

  for _ in range(1000):
    _, _ = cap.read()

  id_scena = 1
  ret, prec = cap.read()
  path = main_dir + '/' + str(id_scena).zfill(3)
  if not os.path.exists(path):
      os.mkdir(path)
  cv2.imwrite(path + '/0.jpg', prec)
  x = int(prec.shape[0] / 5)
  y = int(prec.shape[1] / 5)
  for i in range(1, int(totalFrames)):
      try:
          ret, frame = cap.read()
          prec_res = cv2.resize(prec, (y, x))
          frame_res = cv2.resize(frame, (y, x))
          h1 = dhash.dhash_int(Image.fromarray(prec_res), size=16)
          h2 = dhash.dhash_int(Image.fromarray(frame_res), size=16)

          bit_diff = get_num_bits_different(h1, h2)

          if bit_diff > thresh_rip:
              # threshold trovato sperimentalmente per sep shot = 213
              # usando 205, i falsi positivi aumentavano solo di 5
              # meglio qualche falso pos in piu' che un falso neg
              if bit_diff > 204:
                  id_scena += 1
              path = main_dir + '/' + str(id_scena).zfill(3)
              if not os.path.exists(path):
                  os.mkdir(path)
              frame = cv2.resize(frame, (1280, 720))
              cv2.imwrite(path + '/' + str(i) + '.jpg', frame)

          prec = frame

      except:
          print('frame finale: ', i)
  print(vid_name)

  #########################################################

  ep_path = 'sep_eps/' + vid_name
  scenes = os.listdir(ep_path)
  scenes.sort()

  da_canc = []
  for s in scenes:
      folder = os.path.join(ep_path, s)
      frames_number = os.listdir(folder)
      if len(frames_number)<3:
          da_canc.append(folder)

  for f in da_canc:
      shutil.rmtree(f)

  #########################################################

  dirs = os.listdir('sep_eps')
  triplets = []
  for main_dir in dirs:
    lista_dir = os.listdir(os.path.join('sep_eps', main_dir))
    lista_dir.sort()
    for d in lista_dir:
      dir_path = os.path.join('sep_eps', main_dir, d)
      toAppend = os.listdir(dir_path)
      if len(toAppend)>2:
        files = []
        for f in toAppend:
          files.append(os.path.join(dir_path, f))
        files.sort()
        for i in range(len(files)-2):
          triplets.append([files[i], files[i+1], files[i+2]])
        
  print(len(triplets))

  #########################################################

  # save ds
  for t in triplets:
    save_dir = 'ds/sequences/' + str(fld).zfill(7)
    os.makedirs(save_dir)
    for i in range(3):
      shutil.copyfile(t[i], save_dir + '/frame' + str(i).zfill(1) + '.jpg')
    fld += 1
    
  print(ep_path)
  shutil.rmtree(ep_path)

at_s01e01
771
sep_eps/at_s01e01
at_s01e02
833
sep_eps/at_s01e02
at_s01e03
548
sep_eps/at_s01e03
at_s01e04
497
sep_eps/at_s01e04
sao_e01
2122
sep_eps/sao_e01
Akira
2733
sep_eps/Akira


In [0]:
!rm -rf /content/ds/tri_testlist.txt
!rm -rf /content/ds/tri_trainlist.txt

ds = os.listdir('ds/sequences')
ds.sort()
for idx, folder in enumerate(ds):
  if idx < 150:
    with open('/content/ds/tri_testlist.txt', 'a') as f:
      f.write('/content/ds/sequences/' + folder + "\n")
  else:
    with open('/content/ds/tri_trainlist.txt', 'a') as f:
      f.write('/content/ds/sequences/' + folder + "\n")

In [0]:
!wc -l /content/ds/tri_testlist.txt
!wc -l /content/ds/tri_trainlist.txt

150 /content/ds/tri_testlist.txt
7354 /content/ds/tri_trainlist.txt


In [0]:
%%capture

# dependencies
!pip3 install -U https://download.pytorch.org/whl/cu100/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision==0.2.1
!pip3 install scipy==1.1.0

!git clone https://github.com/simonedeldeo/DAIN.git

%cd DAIN

In [0]:
%%capture

%cd my_package 
!./build.sh
%cd ../PWCNet/correlation_package_pytorch1_0
!./build.sh
%cd ..
%cd ..
%cd MegaDepth
!mkdir checkpoints
%cd checkpoints
!mkdir test_local
%cd test_local
!wget http://vllab1.ucmerced.edu/~wenbobao/DAIN/best_generalization_net_G.pth
%cd ../../../PWCNet
!wget http://vllab1.ucmerced.edu/~wenbobao/DAIN/pwc_net.pth.tar
%cd  ..

In [0]:
# pretrained model
%cd ..
!mkdir model_weights
%cd model_weights
!mkdir DAIN
%cd DAIN
!wget http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
%cd ..
%cd ..

/content
/content/model_weights
/content/model_weights/DAIN
--2020-02-23 01:35:08--  http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
Resolving vllab1.ucmerced.edu (vllab1.ucmerced.edu)... 169.236.184.68
Connecting to vllab1.ucmerced.edu (vllab1.ucmerced.edu)|169.236.184.68|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96319643 (92M)
Saving to: ‘best.pth’

best.pth            100%[===================>]  91.86M  18.3MB/s    in 6.1s    

2020-02-23 01:35:14 (15.1 MB/s) - ‘best.pth’ saved [96319643/96319643]

/content/model_weights
/content


In [0]:
%cd DAIN

/content/DAIN


In [0]:
!CUDA_VISIBLE_DEVICES=0 python train.py --pretrained /content/model_weights/DAIN/best.pth --numEpoch 2 --datasetPath /content/ds --batch_size 1 --save_which 1 --lr 0.0001 --rectify_lr 0.0001 --flow_lr_coe 0.01 --occ_lr_coe 0.0 --filter_lr_coe 1.0 --ctx_lr_coe 1.0 --alpha 0.0 1.0 --patience 4 --factor 0.2

revise the unique id to a random numer 64307
Namespace(SAVED_MODEL='/content/model_weights/DAIN/best.pth', alpha=[0.0, 1.0], arg='/content/drive/My Drive/__progetto int sys/ckps/DAIN/4k_720/64307-Sun-Feb-23-01:35/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='/content/ds', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, log='/content/drive/My Drive/__progetto int sys/ckps/DAIN/4k_720/64307-Sun-Feb-23-01:35/log.txt', lr=0.0001, netName='DAIN', no_date=False, numEpoch=2, occ_lr_coe=0.0, patience=4, rectify_lr=0.0001, save_path='/content/drive/My Drive/__progetto int sys/ckps/DAIN/4k_720/64307-Sun-Feb-23-01:35', save_which=1, seed=1, time_step=0.5, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Turn the model into CUDA
Fine tuning on /content/model_weights/DAIN/best.pth
7502

In [0]:
!cp '/content/model_weights/3857-Sun-Jan-26-15:42/epoch1.pth' '/content/drive/My Drive/__progetto int sys/ckps/DAIN/padx4/'